In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import logging
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

In [2]:
# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("scraper.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [3]:
def extract_table_data(table):
    """Extracts data from an HTML table into a list of rows."""
    data = []
    for row in table.find_all('tr'):
        # Get both header and data cells
        cells = row.find_all(['td', 'th'])
        # Clean up the text from each cell
        row_data = [cell.get_text(strip=True) for cell in cells if cell.get_text(strip=True)]
        if row_data:
            data.append(row_data)
    return data

In [4]:
# --------------------------
# NECTA Results Page Scraping
# --------------------------

def scrape_necta_performance(url):
    """Scrapes the given URL and returns the two performance sections' data."""
    response = requests.get(url)
    if response.status_code != 200:
        return ['TOTAL PASSED CANDIDATES',None],['GPA',None],None,None
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the overall performance section heading and then its table
    overall_heading = soup.find(lambda tag: tag.name in ["h1", "h2", "td", "th"] and "EXAMINATION CENTRE OVERALL PERFORMANCE" in tag.get_text())
    overall_data = None
    if overall_heading:
        overall_table = overall_heading.find_next('table')
        if overall_table:
            overall_data = extract_table_data(overall_table)
    
    # Find the division performance section heading and then its table
    division_heading = soup.find(lambda tag: tag.name in ["h1", "h2", "td", "th"] and "EXAMINATION CENTRE DIVISION PERFORMANCE" in tag.get_text())
    division_data = None
    if division_heading:
        division_table = division_heading.find_next('table')
        if division_table:
            division_data = extract_table_data(division_table)
    #passed student list,gpa list, registered,sat for exam
    return overall_data[0],overall_data[1], division_data[0][0],division_data[0][2]
    

In [5]:
# --------------------------
# School Profile Page Scraping
# --------------------------
def scrape_school_features(url):
    """
    Scrapes the school profile page to extract feature availability from the container
    with class 'listed_features'. It builds a dictionary where each feature name (upper-cased)
    maps to "Available" or "Not Available" based on the icon class.
    """
    features = {}
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')
        features_container = soup.find('div', class_='listed_features')
        if features_container:
            feature_divs = features_container.find_all('div', recursive=False)
            for div in feature_divs:
                span = div.find('span')
                feature_name = span.get_text(strip=True) if span else None
                icon = div.find('i')
                # Check for the icon class: checkmark means available
                status = "Available" if icon and "icon-checkmark" in icon.get("class", []) else "Not Available"
                if feature_name:
                    features[feature_name.upper()] = status
        return features
    except Exception as e:
        print("Error scraping school features:", e)
        return features


In [6]:
def parse_school_profile_features(features):
    """
    Interprets the features dictionary to produce:
      - SCHOOL OWNERSHIP (GOVERNMENT or PRIVATE)
      - SCHOOL CATEGORY (BOYS ONLY, GIRLS ONLY, or BOYS AND GIRLS)
      - SCHOOL TYPE (DAY, BOARDING, or DAY AND BOARDING)
      - ACADEMIC LEVEL CATEGORY (ALEVEL ONLY or COMBINED OA)
    """
    # Determine SCHOOL OWNERSHIP
    if features.get("GOVERNMENT", "Not Available") == "Available":
        school_ownership = "GOVERNMENT"
    elif features.get("PRIVATE", "Not Available") == "Available":
        school_ownership = "PRIVATE"
    else:
        school_ownership = None
    
    # Determine SCHOOL CATEGORY
    if features.get("BOYS ONLY", "Not Available") == "Available":
        school_category = "BOYS ONLY"
    elif features.get("GIRLS ONLY", "Not Available") == "Available":
        school_category = "GIRLS ONLY"
    elif features.get("BOYS & GIRLS", "Not Available") == "Available" or features.get("BOYS AND GIRLS", "Not Available") == "Available":
        school_category = "BOYS AND GIRLS"
    else:
        school_category = None
    
    # Determine SCHOOL TYPE
    day_status = features.get("DAY", "Not Available")
    boarding_status = features.get("BOARDING", "Not Available")
    if day_status == "Available" and boarding_status == "Available":
        school_type = "DAY AND BOARDING"
    elif day_status == "Available":
        school_type = "DAY"
    elif boarding_status == "Available":
        school_type = "BOARDING"
    else:
        school_type = None
    
    # Determine ACADEMIC LEVEL CATEGORY
    # Check for both O-LEVEL and A-LEVEL availability.
    o_level = features.get("O-LEVEL", features.get("OLEVEL", "Not Available"))
    a_level = features.get("A-LEVEL", features.get("ALEVEL", "Not Available"))
    if a_level == "Available" and o_level == "Available":
        academic_level_category = "COMBINED OA"
    elif a_level == "Available":
        academic_level_category = "ALEVEL ONLY"
    else:
        academic_level_category = None
    
    return school_ownership, school_category, school_type, academic_level_category


In [7]:

# --------------------------
# Academics Page Scraping
# --------------------------
def scrape_advanced_level_combinations(url):
    """
    Scrapes the academics page for Advanced Level Combinations.
    Looks for a <div class="sub-comb"> where the header contains "Advanced Level Combinations",
    then collects all combination texts from the <p> tags in the sibling div.
    """
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')
        sub_comb_divs = soup.find_all('div', class_='sub-comb')
        combinations = []
        for sub_comb in sub_comb_divs:
            header_div = sub_comb.find('div', class_='profile_facility_symbol')
            if header_div and "Advanced Level Combinations" in header_div.get_text():
                combination_container = header_div.find_next_sibling('div')
                if combination_container:
                    p_tags = combination_container.find_all('p')
                    for p in p_tags:
                        comb_text = p.get_text(strip=True)
                        if comb_text:
                            combinations.append(comb_text)
                break
        return combinations
    except Exception as e:
        print("Error scraping advanced level combinations:", e)
        return []



In [8]:
def get_combination_category(combination_list):
    """
    Determines the overall combinations category based on the list of combination abbreviations.
    Uses a mapping (e.g., 'PCM' and 'PCB' map to SCIENCE, 'HGL' maps to ARTS).
    Returns a single category or 'MIXED' if multiple categories are found.
    """
    category_mapping = {
        'PCM': 'SCIENCE',
        'PCB': 'SCIENCE',
        'CBG' : 'SCIENCE',
        'PGM' : 'SCIENCE',
        'CBM' : 'SCIENCE',
        'HGL': 'ARTS',
        'HKL' : 'ARTS',
        'HGK' : 'ARTS',
        'HGE' : 'ARTS',
        'ECA' : 'BUSSINESS',
        'ECG' : 'BUSSINESS',
        'EGM' : 'BUSSINESS',
        'HEA' : 'BUSSINESS',
        'ECM' : 'BUSSINESS',
        'PEM' : 'MIXED',
        'GHE' : 'MIXED',
        'PCE' : 'MIXED',
        'CBE' : 'MIXED',
        'MEC' : 'MIXED',
    }
    categories_found = set()
    for comb in combination_list:
        normalized = comb.strip().upper()
        if normalized in category_mapping:
            categories_found.add(category_mapping[normalized])
        else:
            categories_found.add("UNKNOWN")
    if len(categories_found) == 1:
        return categories_found.pop()
    else:
        return "MIXED"


In [9]:
def extract(year=2023,school_no=101):
    # Define URLs
    # necta_url = "https://onlinesys.necta.go.tz/results/2023/acsee/results/s0110.htm"
    # profile_url = "https://www.school.co.tz/s0110/school-profile"
    # academics_url = "https://www.school.co.tz/s0110/academics"

    necta_url = f"https://onlinesys.necta.go.tz/results/2023/acsee/results/s{school_no:04d}.htm"
    profile_url = f"https://www.school.co.tz/s{school_no:04d}/school-profile"
    academics_url = f"https://www.school.co.tz/s{school_no:04d}/academics"
    
    # 1. Scrape NUMBER OF CANDIDATES from NECTA results page
    students,gpa,registered,sat = scrape_necta_performance(necta_url)
    
    # 2. Scrape school features from the profile page and interpret them
    features = scrape_school_features(profile_url)
    school_ownership, school_category, school_type, academic_level_category = parse_school_profile_features(features)
    
    # 3. Scrape Advanced Level Combinations from the academics page and determine its category
    combinations = scrape_advanced_level_combinations(academics_url)
    combinations_category = get_combination_category(combinations) if combinations else None
    
    # Combine all extracted data into a single dictionary
    data = {
        "SCHOOL" : f"s{school_no:04d}",
        "NUMBER OF CANDIDATES": registered,
        "SAT FOR EXAM" : sat,
        "SCHOOL OWNERSHIP": school_ownership,
        "SCHOOL CATEGORY": school_category,
        "SCHOOL TYPE": school_type,
        "COMBINATIONS CATEGORY": combinations_category,
        "ACADEMIC LEVEL CATEGORY": academic_level_category,
        students[0] : students[1],
        "GPA" : gpa[1][:6] if gpa[1] else None
    }
    
    # Create a DataFrame with a single row for this school
    df = pd.DataFrame([data])
    return df

In [10]:
school_range = list(range(100, 6278))

# Use ThreadPoolExecutor for parallel processing
with ThreadPoolExecutor(max_workers=4) as executor:
    # Map the extract function across all schools
    futures = list(tqdm(
        executor.map(lambda x: extract(2023, x).set_index("SCHOOL"), school_range),
        total=len(school_range),
        desc="Scraping Schools"
    ))

# Combine all the dataframes
df = pd.concat(futures)
df

Scraping Schools: 100%|███████████████████| 6178/6178 [1:20:25<00:00,  1.28it/s]


,NUMBER OF CANDIDATES,SAT FOR EXAM,SCHOOL OWNERSHIP,SCHOOL CATEGORY,SCHOOL TYPE,COMBINATIONS CATEGORY,ACADEMIC LEVEL CATEGORY,TOTAL PASSED CANDIDATES,GPA
SCHOOL,,,,,,,,,
s0100,None,None,None,None,None,None,None,None,None
s0101,197,196,GOVERNMENT,BOYS ONLY,DAY AND BOARDING,MIXED,COMBINED OA,196,2.7577
s0102,None,None,None,None,None,None,None,None,None
s0103,471,467,None,None,None,None,None,467,2.7099
s0104,131,131,None,None,None,None,None,130,2.7455
...,...,...,...,...,...,...,...,...,...
s6273,None,None,None,None,None,None,None,None,None
s6274,None,None,None,None,None,None,None,None,None
s6275,None,None,None,None,None,None,None,None,None


In [15]:
# df.dropna(axis=0)

In [16]:
len(df.dropna(axis=0))

174

In [17]:
df.to_csv('project_school_performance_data(2023).csv')

In [18]:
df.dropna(axis=0).to_csv('project_school_performance_data(2023)_.csv')